In [ ]:
import java.sql.Timestamp
case class Coordinates(lat:Double, lon: Double) {
  val R = 6378.137 
  def distanceTo(other: Coordinates): Double = {
    def sqr(x: Double): Double = Math.pow(x,2.0)
    def rad(x: Double): Double = x *  Math.PI / 180     
    val deltaLat = rad(other.lat) - rad(this.lat)
    val deltaLon = rad(other.lon) - rad(this.lon)
    val a = sqr(Math.sin(deltaLat/2)) + Math.cos(rad(this.lat)) * Math.cos(rad(other.lat)) *
      sqr(Math.sin(deltaLon/2))
    val c = 2 * Math.atan2(Math.sqrt(a), Math.sqrt(1-a))
    R * c * 1000// result in meters
  }
}

In [ ]:
case class CarLocation(id:String, timestamp: Timestamp, location: Coordinates)

In [ ]:
case class TimeLocation(time: Timestamp, location: Coordinates) {
  def speedTo(other: TimeLocation): Double = {
    if (other.time == this.time) 0.0 else {
      val distance = location.distanceTo(other.location) // mts
      val deltaT = Math.abs(time.getTime - other.time.getTime) / 1000 // seconds
      distance/deltaT * 3.6 // km/h
    }
  }
}

In [ ]:
case class TripState(id: String, start: TimeLocation, current: TimeLocation, end: Option[TimeLocation] = None)

import java.sql.Timestamp
defined class Coordinates
defined class CarLocation
defined class TimeLocation
defined class TripState


In [ ]:
val bxl = Coordinates(50.8503, 4.3517)
val antwerp = Coordinates(51.2194,4.4025)

bxl: Coordinates = Coordinates(50.8503,4.3517)
antwerp: Coordinates = Coordinates(51.2194,4.4025)


In [ ]:
bxl.distanceTo(antwerp)

res4: Double = 41241.62572838361


41241.62572838361

In [ ]:
val now = System.currentTimeMillis
val bxlPickup = TimeLocation(new Timestamp(now), bxl)
val antwerpDrop = TimeLocation(new Timestamp(now + 30*60*1000), antwerp)

now: Long = 1530738049534
bxlPickup: TimeLocation = TimeLocation(2018-07-04 23:00:49.534,Coordinates(50.8503,4.3517))
antwerpDrop: TimeLocation = TimeLocation(2018-07-04 23:30:49.534,Coordinates(51.2194,4.4025))


In [ ]:
val adrop = antwerpDrop.speedTo(bxlPickup) 

adrop: Double = 22.91201429354645


In [ ]:
adrop * 3.6 //km/h

res19: Double = 82.48325145676722


82.48325145676722

In [ ]:
def updateStateWithEvent(state: TripState, event: CarLocation): TripState = {
  state.current.time.compareTo(event.timestamp) match {
    case 0 =>  // same timestamp => do nothing, return same state
      state
    case -1 => // state.currentTime < event.timestamp => check update of start time
      if (state.start.time.before(event.timestamp)) {
        state.copy(start = TimeLocation(event.timestamp, event.location))
      } else {
        state
      }
    case 1 => // state.current.time > event.timestamp => update state
      val currentTimeLocation = TimeLocation(event.timestamp, event.location)
      if (state.current.speedTo(currentTimeLocation) < 1) { // improve the heuristic here
        state.copy(current = currentTimeLocation, end = Some(currentTimeLocation))
      } else {
        state.copy(current = currentTimeLocation)
      }
  }
}

updateStateWithEvent: (state: TripState, event: CarLocation)TripState


In [ ]:
import org.apache.spark.sql.streaming.{GroupStateTimeout, OutputMode, GroupState}

def updateAcrossEvents(id: String,
    inputs: Iterator[CarLocation],
     oldState: GroupState[CarState]): CarState = {
     val state: CarState = if (oldState.exists) oldState.get else CarState(id,
        "",
        new java.sql.Timestamp(6284160000000L),
        new java.sql.Timestamp(6284160L)
    )
  // we simply specify an old date that we can compare against and
  // immediately update based on the values in our data

  for (input <- inputs) {
    state = updateUserStateWithEvent(state, input)
    oldState.update(state)
  }
  state
}

import org.apache.spark.sql.streaming.{GroupStateTimeout, OutputMode, GroupState}
updateAcrossEvents: (user: String, inputs: Iterator[InputRow], oldState: org.apache.spark.sql.streaming.GroupState[UserState])UserState


In [ ]:
val t1 = new Timestamp(System.currentTimeMillis)

t1: java.sql.Timestamp = 2018-07-04 17:28:28.389


In [ ]:
val t2 = new Timestamp(System.currentTimeMillis) 

t2: java.sql.Timestamp = 2018-07-04 17:28:49.089


In [ ]:
t2.compareTo(t1)

res29: Int = 1


1